In [1]:
import subprocess
import numpy as np
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from astropy.table import Table
from astropy.io import fits
from astropy import wcs
from glob import glob
import shlex
from regions import read_ds9, write_ds9
import numpy.ma as ma

# parallel processor et al.
from utilities import parallel_process, check_exe, system_call, system_call_env
from utilities import get_immediate_subdirectories, redshifts_from_papers
from model import inv_beta_model

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)
from astropy import log
log.setLevel('WARN')

/home/boada/Projects/swiftXRT/utilities.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# this allows us to run the pipeline not interactively. 
os.environ['HEADASNOQUERY'] = ''
os.environ['HEADASPROMPT'] = '/dev/null'
# os.environ['PFILES'] = '/tmp/$$.tmp/pfiles;$HEADAS/syspfiles'

In [3]:
def create_bkg_regions(name, outpath, vtp_regions=[], circ_regions=[]):
    ''' For each obs id we are going to create a background region. 10.5 arcmin
    centered on the pointing position of each observation.

    '''

    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    # find the event files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw[1-4]po_cl.evt',
                 recursive=True)

    if len(files) < 1:
        return

    # create a place for the products
    if not os.path.isdir(f'{outpath}/{name}/spec_files'):
        os.makedirs(f'{outpath}/{name}/spec_files')

    # get the MCMC fits:
    if os.path.isfile(f'{outpath}/{name}/{name}_mcmcfits.txt'):
        mcmcfit = Table.read(f'{outpath}/{name}/{name}_mcmcfits.txt',
                             format='ascii',
                             header_start=0)
    else:
        return    
    
    # write one background region for each observation
    for f_in in files:

        # get pointing
        point_ra = fits.getheader(f_in, ext=0)['RA_PNT']
        point_dec = fits.getheader(f_in, ext=0)['DEC_PNT']

        # we cannot use the X/Y position in the catalog because it was detected 
        # in the combined frame. We need to use the RA/DEC and then find the 
        # pixel positions in the individual frames. Lame.
        # WCS info to convert world coords to pixels
        hdr = fits.getheader(f_in.replace('cl.evt', 'ex.img'))
        w = wcs.WCS(hdr)
        pixels = w.wcs_world2pix(point_ra, point_dec, 0)

        obs_id = f_in.split('/')[-2]

        # write out the regions
        with open(f'{outpath}/{name}/spec_files/{obs_id}_bkg.reg', 'w') as reg:
            reg.write('image\n')
            # write the background region -- different sizes for different types of obs.
            if f_in[-10] == '3':
                reg.write(
                    f"circle({pixels[0]:.5f},{pixels[1]:.5f},{(11 * 60 /2.36):.5f})\n"
                )
            else:
                reg.write(
                    f"circle({pixels[0]:.5f},{pixels[1]:.5f},{(9.5 * 60 /2.36):.5f})\n"
                )

            for j, xc, yc, rc, rotc, ext in detects[['INDEX', 'RA', 'DEC', 'R', 'ROTANG', 'Extended']]:
                
                # gotta work in pixel coordinates
                pixels = w.wcs_world2pix(xc, yc, 0) 
                
                # we didn't fit all the sources with the MCMC -- use the VTP region
                if j not in mcmcfit['ID'] or ext < 1:
                    reg.write(f'-ellipse({pixels[0]:.5f},{pixels[1]:.5f},{rc[0]:.3f}'
                              f',{rc[1]:.3f},{rotc:.3f})\n')
                    continue
                
                # check to see if we WANT vtp regions
                if j in vtp_regions:
                    reg.write(f'-ellipse({pixels[0]:.5f},{pixels[1]:.5f},{rc[0]:.3f}'
                              f',{rc[1]:.3f},{rotc:.3f})\n')
                    continue
                # or we WANT circularized regions
                elif j in circ_regions:
                    radius = max(rc[0], rc[1])
                    reg.write(f"-circle({pixels[0]}, {pixels[1]}, {radius:.3f})\n"
                            )
                    continue
                    
                # Here we are going to draw a circle with a radius where the beta 
                # model just fades into the background. These values come from 
                # the MCMC fits done previously.
                radius = inv_beta_model(mcmcfit['So_50'][mcmcfit['ID'] == j],
                                        mcmcfit['rc_50'][mcmcfit['ID'] == j],
                                        mcmcfit['beta_50'][mcmcfit['ID'] == j],
                                        mcmcfit['bg_50'][mcmcfit['ID'] == j])               
                
                reg.write(
                    f"-circle({pixels[0]}, {pixels[1]}, {(1.5 * radius[0] * 60 / 2.36):.3f})\n"
                )
                


In [4]:
def create_src_regions(name, outpath, vtp_regions=[], circ_regions=[]):
    ''' For each obs id we are going to create a background region. 10.5 arcmin
    centered on the pointing position of each observation.

    '''

    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    # find the event files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw[1-4]po_cl.evt',
                 recursive=True)

    if len(files) < 1:
        return

    # create a place for the products
    if not os.path.isdir(f'{outpath}/{name}/spec_files'):
        os.makedirs(f'{outpath}/{name}/spec_files')

    # get the MCMC fits:
    if os.path.isfile(f'{outpath}/{name}/{name}_mcmcfits.txt'):
        mcmcfit = Table.read(f'{outpath}/{name}/{name}_mcmcfits.txt',
                             format='ascii',
                             header_start=0)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_mcmcfits.txt')

    # write one background region for each observation
    for f_in in files:

        obs_id = f_in.split('/')[-2]

        # we cannot use the X/Y position in the catalog because it was detected 
        # in the combined frame. We need to use the RA/DEC and then find the 
        # pixel positions in the individual frames. Lame.
        # WCS info to convert world coords to pixels
        hdr = fits.getheader(f_in.replace('cl.evt', 'ex.img'))
        w = wcs.WCS(hdr)

        # now we have to loop through the detections
        for j, xc, yc, ext in detects[['INDEX', 'RA', 'DEC', 'Extended']]:

            # we didn't fit all the sources with the MCMC
            if j not in mcmcfit['ID']:
                continue
            elif ext < 1:
                continue
                
            # write out the regions
            with open(f'{outpath}/{name}/spec_files/{obs_id}_{j}.reg',
                      'w') as reg:
                reg.write('image\n')
                    
                # Here we are going to draw a circle with a radius where the beta 
                # model just fades into the background. These values come from 
                # the MCMC fits done previously.
                radius = inv_beta_model(mcmcfit['So_50'][mcmcfit['ID'] == j],
                                        mcmcfit['rc_50'][mcmcfit['ID'] == j],
                                        mcmcfit['beta_50'][mcmcfit['ID'] == j],
                                        mcmcfit['bg_50'][mcmcfit['ID'] == j])

                pixels = w.wcs_world2pix(xc, yc, 0)

                reg.write(
                    f"circle({pixels[0]}, {pixels[1]}, {(radius[0] * 60 / 2.36):.3f})\n"
                )

                # have to loop through them again for the masks
                for k, xc2, yc2, rc, rotc in detects[['INDEX', 'RA', 'DEC', 'R', 'ROTANG']]:
                    
                    if not j == k:
                        
                        pixels = w.wcs_world2pix(xc2, yc2, 0)
                        
                        # we didn't fit all the sources with the MCMC
                        if k not in mcmcfit['ID']:
                            reg.write(f'-ellipse({pixels[0]:.5f},{pixels[1]:.5f},'
                                  f'{rc[0]:.3f},{rc[1]:.3f},{rotc:.3f})\n')
                            continue
                        
                        
                        # check to see if we WANT vtp regions
                        if j in vtp_regions:
                            reg.write(f'-ellipse({pixels[0]:.5f},{pixels[1]:.5f},{rc[0]:.3f}'
                                      f',{rc[1]:.3f},{rotc:.3f})\n')
                            continue
                        # or we WANT circularized regions
                        elif j in circ_regions:
                            radius = max(rc[0], rc[1])
                            reg.write(f"-circle({pixels[0]}, {pixels[1]}, {radius:.3f})\n"
                                    )
                            continue 
                        
                        
                        # Here we are going to draw a circle with a radius where the beta 
                        # model just fades into the background. These values come from 
                        # the MCMC fits done previously.
                        radius = inv_beta_model(mcmcfit['So_50'][mcmcfit['ID'] == k],
                                                mcmcfit['rc_50'][mcmcfit['ID'] == k],
                                                mcmcfit['beta_50'][mcmcfit['ID'] == k],
                                                mcmcfit['bg_50'][mcmcfit['ID'] == k])

                        reg.write(
                            f"-circle({pixels[0]}, {pixels[1]}, {(1.5 * radius[0] * 60 / 2.36):.3f})\n"
                        )

In [5]:
outpath = './data'
name = 'PSZ1_G292.00-43.64'
vtp_regions = [9, 11, 12, 14, 15, 16, 23, 26, 29]
create_src_regions(name, outpath, vtp_regions=vtp_regions)

name = 'PSZ2_G052.35-31.98'
vtp_regions = [1, 2]
create_src_regions(name, outpath, vtp_regions=vtp_regions)

name = 'PSZ2_G226.75+48.95'
vtp_regions = [3,4,6]
create_src_regions(name, outpath, vtp_regions=vtp_regions)

name = 'PSZ2_G230.29-47.13'
circ_regions = [1, 2]
create_src_regions(name, outpath, circ_regions=circ_regions)

name = 'PSZ2_G078.36-47.22'
vtp_regions = [1, 2]
create_src_regions(name, outpath, vtp_regions=vtp_regions)

name = 'PSZ2_G162.49-71.98'
circ_regions = [1, 2]
create_src_regions(name, outpath, circ_regions=circ_regions)




/home/boada/Projects/swiftXRT/model.py:52: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(c)
